In [15]:
import math
import random
import functools
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd


In [318]:
def createRoute(routeNodes): #retorna uma rota embaralhada.
    '''
    Shuffles randomly the given array
    '''
    route = random.sample(routeNodes, len(routeNodes))
    return route


def initialPopulation(routeNodes, size=100): #gera uma população. Cada posicao é uma rota diferente.
    '''
    Generate initial population by randomly shuffling the route nodes.
    Default population size: 100
    '''
    population = []
    for i in range(0, size):
        population.append(createRoute(routeNodes))
    return population


def distances_between_points(p1, p2):
    '''
    Returns the square distance between two given points
    '''
    return np.dot((p1-p2),(p1-p2))

def fitness(route, cities):
    '''
    Calculates and return route fitness with base in the Euclidean distance.
    
    The fitness is given by:
        1 / log10(Euclidean distance)
    '''
    distances_between_cities = []
    
    for i in range(1, len(route)):
        if i != len(route): # distance between cities along route
                                            
            p1 = np.array(cities['COORD'][route[i-1]],cities['SECTION'][route[i-1]]) #segundo ponto
            p2 = np.array(cities['COORD'][route[i]],cities['SECTION'][route[i]]) # ultimo ponto - 1.
            distances_between_cities.append(distances_between_points(p1,p2))
        else: # distance between first and last city
            p1 = np.array(cities['COORD'][route[0]],cities['SECTION'][route[0]]) #primeiro ponto
            p2 = np.array(cities['COORD'][route[i-1]],cities['SECTION'][route[i-1]]) #ultimo ponto
            distances_between_cities.append(distances_between_points(p1,p2))
        
    euclidan_distance = math.sqrt(np.sum(distances_between_cities))
    
    fitness = 1 / np.log10(euclidan_distance)
    
    return fitness

def orderedCrossover_v1(parent_1, parent_2):
    '''
    
    A version of ordered crossover that returns a valid child for TSP
    This implementation works that way:
    
    Consider the parents:
        parent_1 = [1, 2, 3, 4, 5, 6, 7, 8, 9]
        parent_2 = [1, 3, 5, 7, 9, 2, 4, 6, 8]
        
    Randomly selects a subset of parent_1:
        parent_1 = [1, 2, | 3, 4, 5, 6, | 7, 8, 9]
    Then creates a 'offspring':
        offspring = [x, x, 3, 4, 5, 6, x, x, x ]
    And fill the Xs with cities of parent_2, in order they appear and without
    duplicating any city:
        offspring = [1, 7, 3, 4, 5, 6, 9, 2, 8]
    '''
    
    gene_1 = 0
    gene_2 = 0
    
    # Takes at least 40% and at most 70% of cities in the first parent
    while abs(gene_1-gene_2) < int(len(parent_1)*0.4):
        gene_1 = np.random.randint(0, int(len(parent_1)*0.7)+1)
        gene_2 = np.random.randint(0, int(len(parent_1)*0.7)+1)
    
    # Selects first and last positions of subset
    first_position = min(gene_1, gene_2)
    last_position = max(gene_1, gene_2)
    
    # Creates a offspring without repetition
    child = []
  
    for i in range(len(parent_2)):
        if parent_2[i] not in parent_1[first_position:last_position+1]:
            child.append(parent_2[i])
        if i in range(first_position, last_position+1):
            child.insert(i,parent_1[i])
    return child 

    
def orderedCrossover_v2(parent_1, parent_2, random_qnt):
    '''
    A version of ordered crossover that returns a valid child for TSP.
        
    '''
    random_positions = random.sample(range(len(parent_1)), random_qnt)
    random_positions.sort()
    
    subset = []
    for i in range(len(random_positions)):
        subset.append(parent_1[random_positions[i]])
    
    positions_ordered = []
    for node in parent_2:
        if node in subset:
            positions_ordered.append(parent_2.index(node))
    
    child = parent_1
    for i in range(len(subset)):
        child[random_positions[i]] = subset[positions_ordered[i]]
    
    return child

def alternativeCrossover(parent_1,parent_2):
      
    child = parent_1
    cut = random.randrange(1,len(parent_1)+1)

    
    for i in range(cut, len(parent_1)):
        if(parent_1[i]) not in parent_2[0:i] and (parent_2[i] not in parent_1[0:i]):
            child[i] = parent_2[i]
    
    return child

def mutate_v1(child): #swap mutation, troca de lugares dois genes
    index_1 = random.randrange(0,len(child)) #pega uma posicao aleatoria
    index_2 = random.randrange(0,len(child)) # pega outra posicao aleatoria

    
    if index_1 != index_2:
        aux = child[index_1]
        child[index_1] = child[index_2]
        child[index_2] = aux
    else:                                        #caso  posicoes sejam iguais
        mutate_v1(child)
    return child

def mutate_v2(child): #Scramble mutation, um conjunto variavel de genes é escolhido, embaralhados, e reinseridos no individuo.

    aux_child = child
    index_1 = random.randrange(0,len(child)) #posicao inicial de split aleatoria
    index_2 = random.randrange(0,len(child)) #posicao final de split aleatoria p

    if (index_1 > index_2) or ((index_2 - index_1) < 1):
        mutate_v2(child)
    
    else:#garante que a fracao cortada seja totalmente diferente da nova  a ser encaixada.
        aux = child[index_1:index_2+1]
        aux1 = aux
        aux = random.sample(aux,len(aux))
        if aux == aux1:
            while aux == aux1:
                aux = random.sample(aux,len(aux))                

        j = 0
        for i in range(index_1,(index_2+1)):
            child[i] = aux[j]
            j = j+1
   
    
    return child 

def acumulate(v): #cria a tabela de distribuicao acumulada
    acum = 0
    r = []
    for i in v:
        acum += i
        r.append(acum)
    return r   


def pick_parents(pop,f,cities):#populacao,funcao de fitness,espaco de estados
  
    fits = []


    for i in range(len(pop)):
        print(pop[i])
    
        fits.append(fitness(pop[i],cities))
    
        
    
    print(fits)
    fits_sum = sum(fits)
    print(fits_sum)

    norm = map(lambda x: x / fits_sum, fits)
    acum = acumulate(norm)
    print(acum)


    r = np.random.uniform()
    print(r)
    
    
    for i in range(len(acum)):
        if r < acum[i]:
            break

    print(pop[i])
    return pop[i]



def main():
    
    cities = pd.read_csv('a280.csv', ';') #le o arquivo csv
    pop = initialPopulation(list(cities['NODE']), 9) #cria a populacao inicial
    pop = np.subtract(pop, 1) 
    pick_parents(pop,fitness,cities)

    

In [326]:
main()

[237 196 174 188 224  64 223 182 227 218  29 168 112 230 206 261 219 266
 201  14 143 187 127 103 255 198  88 208 260 171 234  53 158 121  75  96
 184 251 169 163  33 193 254  15 211 256 203 238  25  72 215  50 249 118
 151  60 178 181 221 205  12  63 233 146 140  26  18  37  24 214  48 175
  42  83 262 229 269  55 259 200 197  28  49 277 152 106  79 275  90 108
 111  44 210 270  98 273 172 192  16 217 123 180 113  70 235 141 149   7
 155   3 156 100  54  30 144 207 276 132  80  10  81  67 110 271  94  13
 236 243  73 247  41  51 136  89 120 105 102  56 179  62 242 116   6 165
 134 142  74  99 244 195  58  35 176 107 160   5 159 130  71 245 246  23
 241 101 264 226 231 213  27  78  61  69   8 157 122  86 154 115   2 185
 257 124  40 220 194  77 139  47  45  32  92 204 248 222  22 240 199  82
  76  87  68 117  85 150 133 258  57  34 209  39  31  20 278 161   1 129
  11  93 191  19 167 202  21 212  97   9  17 138 137 232 228 109 183 126
  52 239 135  65 250 147  36  84 128 148 216 166  9